In [5]:
import pickle as pkl
import spacy
from tqdm import tqdm

In [6]:
import csv
import json

In [7]:
sm_parser = spacy.load('en_core_web_sm')

In [8]:
with open('tbbt_en_zh.pkl', 'rb') as f_zh:
    with open('tbbt_en_fa.pkl', 'rb') as f_fa:
        zh = pkl.load(f_zh)
        fa = pkl.load(f_fa)
        inter_keys = set(zh.keys()) & set(fa.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'tbbt_en_zh.pkl'

In [4]:
data = {}
with open('parsed_corpus.pkl', 'rb') as f:
    parsed = pkl.load(f)
    for item in inter_keys:
        data[item] = parsed[item]

NameError: name 'inter_keys' is not defined

In [144]:
count = 0
for epi_key in tqdm(list(data.keys())):
    episode = data[epi_key]
    for scene in episode:
        candidate_spans = []
        query_spans = []
        sentences = []
        for i, utt in enumerate(scene):
            if 'en_subtitles' in utt:
                berkeley_noun_chunk = utt['berkeley_noun_chunk']
                sm_noun_chunk = utt['sm_noun_chunk']
                trf_noun_chunk = utt['trf_noun_chunk']

                nps = list(set(set(berkeley_noun_chunk) | set(sm_noun_chunk) | set(trf_noun_chunk)))
                count += len(nps)
print(count)

100%|██████████| 96/96 [00:00<00:00, 2223.06it/s]

71194


In [121]:
to_annotate_output = []

for epi_key in tqdm(list(data.keys())[:20]):
    episode = data[epi_key]
    for scene in episode:
        candidate_spans = []
        query_spans = []
        sentences = []
        for i, utt in enumerate(scene):
            if 'en_subtitles' in utt:
                utterance = " ".join([x.strip().lstrip('-').lstrip().lstrip('.').lstrip() for x in utt['en_subtitles']])
                utterance_tokens = [item.text for item in sm_parser(utterance)]
                speaker = utt['speaker']
                utterance_tokens.insert(0, ":")
                utterance_tokens.insert(0, speaker)
                sentences.append(utterance_tokens)

                candidate_spans.append(
                    {
                        "sentenceIndex": i,
                        "startToken": 0,
                        "endToken": 1
                    }
                )

                berkeley_noun_chunk = utt['berkeley_noun_chunk']
                sm_noun_chunk = utt['sm_noun_chunk']
                trf_noun_chunk = utt['trf_noun_chunk']

                nps = list(set(set(berkeley_noun_chunk) | set(sm_noun_chunk) | set(trf_noun_chunk)))
                nps.sort(key=lambda x:x[1])
                for np in nps:
                    span = {
                        "sentenceIndex": i,
                        "startToken": np[1]+2,
                        "endToken": np[2]+2
                    }
                    candidate_spans.append(span)
                    query_spans.append(span)
        to_annotate_output.append({
                "sentences": sentences,
                "querySpans": query_spans,
                "candidateSpans": candidate_spans
        })

100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


In [122]:
print(len(to_annotate_output))

26


In [138]:
for item in to_annotate_output[0]:
    print(item)
    print(to_annotate_output[0][item])
    print("=="*50)

sentences
[['Leonard', ':', 'So', 'are', 'you', 'really', 'not', 'gon', 'na', 'do', 'this', 'movie', '?'], ['Penny', ':', 'Well', ',', 'I', 'do', "n't", 'think', 'it', "'s", 'the', 'kind', 'of', 'part', 'that', "'s", 'good', 'for', 'my', 'career', '.'], ['Leonard', ':', 'Well', ',', 'but', 'do', "n't", 'a', 'lot', 'of', 'famous', 'actors', 'get', 'their', 'start', 'doing', 'bad', 'movies', '?'], ['Penny', ':', 'Okay', ',', 'I', 'do', "n't", 'think', 'Meryl', 'Streep', 'ever', 'had', 'to', 'say', '...', '"', 'Must', 'keep', 'gorilla', 'hands', 'from', 'killing', 'again', '!', '"'], ['Raj', ':', 'If', 'she', 'did', ',', 'it', 'would', 'be', 'amazing', '.', 'That', 'woman', 'can', 'do', 'no', 'wrong', '.'], ['Leonard', ':', 'I', '-', 'I', 'do', "n't", 'know', 'anything', 'about', 'show', 'business', ',', 'so', 'if', 'you', 'think', 'it', "'s", 'reasonable', 'to', 'turn', 'down', 'paid', 'work', 'and', 'just', 'burn', 'through', 'your', 'savings', ',', 'I', "'m", 'sure', 'you', 'know', 'wh

In [126]:
def write_annotation_file(output, file_path):
    with open(file_path, "w", encoding="utf-8") as csv_fh:
        fieldnames = ['json_data']
        writer = csv.DictWriter(csv_fh, fieldnames, lineterminator='\n')
        writer.writeheader()
        for line in output:
            writer.writerow({'json_data': json.dumps(line)})

In [129]:
write_annotation_file(to_annotate_output[:20], "test_merged.csv")

In [63]:
for instance in data[(1,1)]:
    for item in instance:
        for x in item:
            print(x, item[x])
        print('=='*50)

utterance  So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.
speaker Sheldon
en_subtitles ['If a photon is directed through a plane with two slits in it and either is observed it will not go through both.', 'If unobserved, it will.', "If it's observed after it left the plane, before it hits its target...", 'it will not have gone through both slits.']
zh_subtitles ['将光子正对平面上的双缝 观察任意一个隙缝 它不会穿过那两个隙缝', '如果没被观察 那就会', '总之 如果观察它在离开平面到击中目标之前', '它就不会穿过那两个隙缝']
sm_noun_chunk [('a photon', 1, 3), ('a plane', 6, 8), ('two slits', 9, 11), ('it', 12, 13), ('it', 17, 18), ('both', 22, 23), ('it', 27, 28), ('it', 31, 32), ('it', 35, 36), ('the plane', 37, 39), ('it', 41, 42), ('its target', 43, 45), ('it', 46, 47), ('both slits', 52, 54)]
sm_pron [('it', 12, 13), ('it', 17, 18

In [49]:
# sentence = "  - Oh, take your time. I'll just finish my crossword puzzle."
sentence = "- ...it left the plane, before it hits its target..."
print(sentence)
print(sentence.strip('-'))
print(sentence.strip().strip('-').strip().strip('.').strip())

- ...it left the plane, before it hits its target...
 ...it left the plane, before it hits its target...
it left the plane, before it hits its target


In [18]:
for item in data[(1,1)][0]:
    for x in item:
        print(x, item[x])
    print('=='*50)

utterance  So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.
speaker Sheldon
en_subtitles ['If a photon is directed through a plane with two slits in it and either is observed it will not go through both.', 'If unobserved, it will.', "If it's observed after it left the plane, before it hits its target...", '- ...it will not have gone through both slits.']
zh_subtitles ['将光子正对平面上的双缝 观察任意一个隙缝 它不会穿过那两个隙缝', '如果没被观察 那就会', '总之 如果观察它在离开平面到击中目标之前', '它就不会穿过那两个隙缝']
sm_noun_chunk [('a photon', 3, 5), ('a plane', 8, 10), ('two slits', 11, 13), ('it', 14, 15), ('either slit', 16, 18), ('it', 20, 21), ('both slits', 25, 27), ('it', 29, 30), ('it', 32, 33), ('it', 38, 39), ('it', 42, 43), ('the plane', 45, 47), ('it', 49, 50), ('its target', 51, 53), ('it', 54, 55), ('both s

In [9]:
print(len(data[(1,1)]))

11
